TensorFlow
- python vesrion 3.8 or 3.11이면 대게 문제가 다 해결됨.
- 데이터의 특성 파악이 제일 중요.

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

MNIST data set:
- x: np.array. 28x28 흑백이미지
- y: 0-9(int)

In [2]:
mnist = tf.keras.datasets.mnist
mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


((array([[[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         ...,
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0

In [1]:
# 데이터세트를 불러옵니다.
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

NameError: name 'mnist' is not defined

In [20]:
x_train.max()

1.0

train 데이터를 나누기 255 하는 이유:
- 0과1로 제한하기 위해서
- 무한대로 발산, +,-로 발산, 수렴하는 것을 방지하기 위해서

test 데이터를 255로 나누지 않는 이유:
- 분류데이터 이기 때문에 맞지 않음.
- 검증하는 데이터기 때문에 학습에 관여 하지 않음.

다중 클래스 분류를 위한 sequential 모델 생성:
- Flatten layer: input shape를 x_train의 shape와 같이 설정
- Dense layer: 레이어를 통과한 값이 어떤 shape로 나올 것인지에 따라 값을 조절. activatein funtion을 데이터 특성에 맞게 조절
- dropout layer: 과적합 방지를 위해, 학습 단계마다 전체 모델의 일정 비율을 비활성화. 마지막 Dense layer: 원하는 분류 클래스의 개수에 따라 값을 지정. 
- 분류 모델의 경우 주로 softmax activation function을 이용

#### 과적합 
- 데이터를 딥러닝 모델이 학습할 때, 학습하는 데이터의 복잡도에 비해서 모델의 복잡도가 너무 높은것 
##### 해결방안:
1. 모델의 복잡도를 낮춘다.
2. 모델의 깊이를 늘린다.
3. 노이즈를 추가한다.

In [21]:
# 모델 학습하기
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)), # flatten layer(1차원으로 평탄화 작업) 데이터의 shape를 정확하게 맞춰서 넣어야함.
  tf.keras.layers.Dense(128, activation='relu'), # 28*28 길이는 784인걸 길이가 128로 출력한다. shape를 조정해줌, Relu는 활성화 함수
  tf.keras.layers.Dropout(0.2), # 노드를 끄는 비율 조절해서 특정값에 수렴하는것을 방지함(과적합).
  tf.keras.layers.Dense(10, activation='softmax') #마지막 Dense는 클래스의 숫자에 맞춰줘야함. 클래스가 다중분류이기 때문에 softmax 사용, 이진일때 sigmoid사용
])

# 컴파일 단계에서는 옵티마이저, 손실함수, 평가기준을 선택
model.compile(optimizer='adam', # 최적화 함수를 어떤걸 사용할 것인가.
              loss='sparse_categorical_crossentropy'# 다중분류,손실 함수를 어떤걸 사용 할 것인가, 이진 분류일때는 binary crossentropy사용. 
              metrics=['accuracy']) # 평가 기준을 무엇으로 할 것인가.

c:\ProgramData\anaconda3\envs\DL\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


## 모델에 이용된 함수를 개별적으로 실행 

In [22]:
model

<Sequential name=sequential, built=True>

In [23]:
# 값 예측
predictions = model(x_train[:1]).numpy()
predictions

array([[0.09340233, 0.09170026, 0.10030396, 0.0570889 , 0.20504922,
        0.13916838, 0.06355625, 0.06893743, 0.09390407, 0.08688916]],
      dtype=float32)

In [25]:
# tf.nn.softmax 함수는 각 클래스에 대한 정답일 확률로 변환합니다.
tf.nn.softmax(predictions).numpy()

array([[0.0992561 , 0.09908731, 0.0999435 , 0.09571642, 0.11098003,
        0.10390421, 0.09633746, 0.09685726, 0.09930591, 0.09861173]],
      dtype=float32)

In [26]:
# 손실값 계산
# 손실값의 경우, 0 이상의 실수로 표현됨. 0에 가까울수록 좋은 결과
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss_fn(y_train[:1], predictions).numpy()

## 모델 학습

In [28]:
# 모델 훈련 및 평가
model.fit(x_train, y_train, epochs=10)

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 835us/step - accuracy: 0.8560 - loss: 0.4858
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 823us/step - accuracy: 0.9556 - loss: 0.1509
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 820us/step - accuracy: 0.9695 - loss: 0.1061
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 824us/step - accuracy: 0.9738 - loss: 0.0876
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 820us/step - accuracy: 0.9777 - loss: 0.0709
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 821us/step - accuracy: 0.9799 - loss: 0.0623
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 824us/step - accuracy: 0.9819 - loss: 0.0551
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 828us/step - accuracy: 0.9827 - loss: 0.0500
Epoch 9/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 825us/step - accuracy: 0.9845 - loss: 0.0471
Epoch 10/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 825us/step - accuracy: 0.9880 - loss: 0.0382


In [29]:
# 테스트 세트를 확인합니다.
# 여러 검증 데이터를 이용해서 정확도 측정
model.evaluate(x_test,  y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 514us/step - accuracy: 0.9756 - loss: 0.0834


[0.07060650736093521, 0.9785000085830688]

In [30]:
# 개별 데이터를 이용해 분류 결과 확인
model.predict(x_train[19:20])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


array([[5.9462217e-13, 1.7483214e-07, 3.4370971e-14, 4.6351008e-04,
        1.0631226e-04, 1.7153690e-05, 3.9189949e-15, 5.6186567e-05,
        3.1406234e-06, 9.9935359e-01]], dtype=float32)

In [31]:
y_train[19]

9